In [1]:
import pandas as pd
from Bio import SeqIO
from io import StringIO
import requests
import esm
from trainer import reshape_features
import torch
from sklearn.metrics import classification_report
from keras.models import load_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tool import model as md
from tool.att import Attention
from tool import config as cfg
import numpy as np
from tqdm import tqdm
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
from featurizer import set_model, get_rep_seq
model,batch_converter,alphabet = set_model()

2024-02-22 13:25:32.302124: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-22 13:25:32.302582: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-22 13:25:32.302725: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-22 13:25:32.332549: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-22 13:25:34.598076: W tensorflow/c

Using device: cuda


In [2]:
data = pd.read_csv("./DATA/openset.csv")
data = data[data["seq"]!="-"]
data = data[data["comple_portal_num"].isin([1,2,3,4,5,6,7,8,10,12])]
data.comple_portal_num.value_counts()
data.seq = data.seq.apply(lambda x:x[:1000])
df_data = list(zip(data.index,data.seq))

In [3]:
# # Embedding
# stride = 20
# num_iterations = len(df_data) // stride
# if len(df_data) % stride != 0:
#     num_iterations += 1
    
# all_results = pd.DataFrame()
# all_emb = []
# for i in tqdm(range(num_iterations)):
#     start = i * stride
#     end = start + stride

#     current_data = df_data[start:end]
#     rep33 = get_rep_seq(current_data,model,batch_converter,alphabet)
#     all_emb.append(rep33.values)
# data['emb'] = np.concatenate(all_emb).tolist()

In [4]:
# data.to_feather("DATA/openset_with_feat.feather")

In [5]:
data

,entry,comple_portal_num,uniprot_num,Subunit_structure,seq
1,A0A1D5P2B5,1,2,SUBUNIT: Homodimer. Heterodimer; with a rar mo...,MDTKHFLPLDFSNQVNSTSLNSPTSRGPMATPSLHPSIGPGIGSSL...
2,A0A287A1S6,2,-,-,MFSFVDLRLLLLLAATALLTHGQEEGQEEGQQGQEEDIPPVTCVQN...
3,A0A2P9E102,2,-,-,MMPHPIQDAVLRVDRLSVVYPGGVTALRDTSIAFRRGEFTVLLGLS...
4,A0A5B9,1,0,Alpha-beta TR is a heterodimer composed of an ...,DLKNVFPPKVAVFEPSEAEISHTQKATLVCLATGFYPDHVELSWWV...
5,A0A8C0MYN9,3,-,-,MTSFVQKGTWLLLALLQPAVISAQQQAIDGGCSHLGQSYADRDVWK...
...,...,...,...,...,...
3669,Q9Z266,1,0,Component of the biogenesis of lysosome-relate...,MAAAGSAAVSGAGTPVAGPTGRDLFAEGLLEFLRPAVQQLDSHVHA...
3670,Q9Z321,1,-,-,MKTVLMVAEKPSLAQSIAKILSRGNMSSHKGLNGACSVHKYTGTFA...
3671,Q9ZZW7,1,0,Forms a ribonucleoprotein complex composed of ...,MAFRKSNVYLSLVNSYIIDSPQPSSINYWWNMGSLLGLCLVIQIVT...
3672,V6CLJ5,1,0,Component of a complex consisting of at least ...,MFSWRRGDCESQKEENRSEERKGEETIRFPTRSPFHCVLFLLTDGF...


#### Train

In [6]:
# Load and preprocess the dataset.
dataset = pd.read_feather("DATA/openset_with_feat.feather")
dataset['label'] = LabelEncoder().fit_transform(dataset['comple_portal_num'])
train_data, vali_data = train_test_split(dataset, test_size=cfg.TRAIN_TEST_SPLIT_SIZE,stratify=dataset['label'])

X_train = reshape_features(pd.DataFrame(train_data['emb'].tolist()))
X_val = reshape_features(pd.DataFrame(vali_data['emb'].tolist()))

print(f'X_train shape: {X_train.shape}')
print(f'X_val shape: {X_val.shape}')

# Train the model.
gru_attention_model = md.GRUWithAttentionModel(input_shape=cfg.INPUT_SHAPE, num_classes=cfg.NUM_CLASSES)
gru_attention_model.compile_model()

history = gru_attention_model.train(
    X_train, 
    train_data['label'],
    X_val, 
    vali_data['label'],
    batch_size=cfg.BATCH_SIZE, 
    epochs=cfg.EPOCHS
)

X_train shape: (2902, 1, 1280)
X_val shape: (726, 1, 1280)


2024-02-22 13:26:22.829941: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10565 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:73:00.0, compute capability: 8.6
2024-02-22 13:26:23.367530: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


Epoch 1/200


2024-02-22 13:26:33.041649: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8700
2024-02-22 13:26:33.461063: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fdf9c00df30 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-02-22 13:26:33.461101: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA RTX A6000, Compute Capability 8.6
2024-02-22 13:26:33.469446: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-02-22 13:26:33.715984: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


2/2 [==============================] - 12s 1s/step - loss: 2.1270 - accuracy: 0.3022 - val_loss: 1.4922 - val_accuracy: 0.7534
Epoch 2/200
2/2 [==============================] - 0s 118ms/step - loss: 1.4779 - accuracy: 0.7536 - val_loss: 1.1201 - val_accuracy: 0.7534
Epoch 3/200
2/2 [==============================] - 0s 115ms/step - loss: 1.1400 - accuracy: 0.7536 - val_loss: 1.0194 - val_accuracy: 0.7534
Epoch 4/200
2/2 [==============================] - 0s 133ms/step - loss: 1.0325 - accuracy: 0.7536 - val_loss: 0.9900 - val_accuracy: 0.7534
Epoch 5/200
2/2 [==============================] - 0s 65ms/step - loss: 0.9879 - accuracy: 0.7536 - val_loss: 0.9432 - val_accuracy: 0.7534
Epoch 6/200
2/2 [==============================] - 0s 98ms/step - loss: 0.9238 - accuracy: 0.7536 - val_loss: 0.8846 - val_accuracy: 0.7534
Epoch 7/200
2/2 [==============================] - 0s 59ms/step - loss: 0.8790 - accuracy: 0.7536 - val_loss: 0.8597 - val_accuracy: 0.7534
Epoch 8/200
2/2 [=============

In [7]:
val_predictions = gru_attention_model.model.predict(X_val, batch_size=cfg.BATCH_SIZE)
ground_truth_labels = vali_data['label'].values
predicted_labels = np.argmax(val_predictions, axis=1)
report = pd.DataFrame(classification_report(ground_truth_labels, predicted_labels, zero_division=0, output_dict=True, target_names=[1,2,3,4,5,6,7,8,10,12])).T
report

1/1 [==============================] - 1s 1s/step


,precision,recall,f1-score,support
1,0.853420,0.957952,0.902670,547.000000
2,0.650602,0.446281,0.529412,121.000000
3,0.857143,0.545455,0.666667,22.000000
4,0.875000,0.437500,0.583333,16.000000
5,1.000000,0.500000,0.666667,4.000000
6,1.000000,0.444444,0.615385,9.000000
7,0.000000,0.000000,0.000000,1.000000
8,1.000000,0.500000,0.666667,2.000000
10,0.000000,0.000000,0.000000,1.000000
12,0.000000,0.000000,0.000000,3.000000


#### Transfer

In [8]:
# Replace 'your_model_path.h5' with the actual path to your saved model file
loaded_model = load_model("./model/deepsub.h5",custom_objects={"Attention": Attention},compile=False)
dataset = pd.read_feather("DATA/openset_with_feat.feather")
dataset['label'] = LabelEncoder().fit_transform(dataset['comple_portal_num'])
X_test = reshape_features(pd.DataFrame(dataset['emb'].tolist()))
predicted = loaded_model.predict(X_test)
predicted_labels = np.argmax(predicted, axis=1)

114/114 [==============================] - 1s 4ms/step


In [9]:
report = pd.DataFrame(classification_report(dataset.label.values, predicted_labels, zero_division=0, output_dict=True, target_names=[1,2,3,4,5,6,7,8,10,12])).T
report

,precision,recall,f1-score,support
1,0.847458,0.256035,0.393258,2734.000000
2,0.183135,0.711921,0.291328,604.000000
3,0.225806,0.392523,0.286689,107.000000
4,0.132653,0.166667,0.147727,78.000000
5,0.153846,0.300000,0.203390,20.000000
6,0.096774,0.266667,0.142012,45.000000
7,0.000000,0.000000,0.000000,7.000000
8,0.000000,0.000000,0.000000,12.000000
10,1.000000,0.400000,0.571429,5.000000
12,0.500000,0.062500,0.111111,16.000000
